In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install wandb

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-zu2v4qyr
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-zu2v4qyr
  Resolved https://github.com/huggingface/transformers to commit 799cea64ac1029d66e9e58f18bc6f47892270723
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
ERROR: Could not find a version that satisfies the requirement huggingface-hub<1.0,>=0.10.0 (from transformers) (from versions: 0.0.1, 0.0.2, 0.0.3rc1, 0.0.3rc2, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.1.0, 0.1.1, 0.1.2, 0.2.0, 0.2.1, 0.4.0)
ERROR: No matching distribution found for huggingface-hub<1.0,>=0.10.0
You should consider upgrading via the '/home/ecbm4040/envTF24/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ecbm4040/envTF24/bin/python3 -m 

In [2]:
!pip install torch transformers wandb -qqq

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            TextDataset, DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
from transformers import EarlyStoppingCallback
# from huggingface_hub.hf_api import HfAp
# import api.filter


import os
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import torch
import pathlib
import random



import wandb
wandb.init(project="my-awesome-project")

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: qulxis. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [2]:
#Tokenize:
file_path =  "data/special_token_versions/textfiles/all_data.txt"  
use_special_tokens = True # use custom tokens
special_tokens_manual= ['<|sensander|>', '<|paddingtonbear|>', '<|hankgreen|>', '<|joerogan|>', '<|elonmusk|>', '<|polite|>', '<|impolite|>','<|neutral|>']
#read new tokens:
token_file_path = "data/special_token_versions/keys.csv"
new_special_tokens = pd.read_csv(token_file_path)["Keys"].values.tolist() + special_tokens_manual #add these to special tokens
special_tokens_dict = {'additional_special_tokens': new_special_tokens} # use for direct add
wandb.run.name = file_path
"""
Here, we specify what we train we use to train our data. In our case, we
have several special tokens (ones that shouldn't be split). 

We include subject tokens, user tokens, and politeness tokens manually in special_tokens_manual

"""

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

#add special tokens:
if use_special_tokens:
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict) #adds special tokens
    model.resize_token_embeddings(len(tokenizer))

block_size = tokenizer.model_max_length
train_dataset = TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=block_size, overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#setup wanbi:
%env WANDB_PROJECT=tweet_analysis

wandb.run.name = file_path
wandb.run.save()

/home/ecbm4040/envTF24/lib/python3.6/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (653573 > 1024). Running this sequence through the model will result in indexing errors
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


env: WANDB_PROJECT=tweet_analysis


True

In [3]:
!nvidia-smi

Mon Dec 12 07:02:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    70W /  70W |    207MiB / 15109MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#Train:
#Largely taken from: https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-

ALLOW_NEW_LINES = False     # seems to work better <--- from source
LEARNING_RATE = 1.372e-4
EPOCHS = 4
seed = random.randint(0,2**32-1)


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



training_args = TrainingArguments(
    report_to="wandb",
    output_dir="./model_files",
    overwrite_output_dir=True,
    do_train=True,
    evaluation_strategy = 'steps',# num_train_epochs=1, #new
    eval_steps = 50, #new
    per_device_train_batch_size=1,
    prediction_loss_only=True,
    logging_steps=5,
    save_steps=0,
    seed=seed,
    learning_rate = LEARNING_RATE,
    metric_for_best_model = 'f1',#new
    load_best_model_at_end = True, #new
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics, #new
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]) #new
#LR schedule stuff?
train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader)
trainer.create_optimizer_and_scheduler(num_train_steps)
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps)

# p_start, p_end = 0.4, 0.9
# def progressify(f):
#     "Control progress bar when calling f"
#     def inner(*args, **kwargs):
#         if trainer.state.epoch is not None:
#             # we only have one epoch, EPOCHS is built into dataset
#             progress.value = p_start + trainer.state.epoch * (p_end - p_start)
#         return f(*args, **kwargs)
#     return inner

# trainer.training_step = progressify(trainer.training_step)



/home/ecbm4040/envTF24/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [5]:
trainer.train()
wandb.finish()


***** Running training *****
  Num examples = 638
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 638
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
5,2.940300
10,2.019000
15,2.210700
20,2.149800
25,1.923000
30,2.112700
35,1.941300
40,1.770300
45,1.959900
50,1.804100




Training completed. Do not forget to share your model on huggingface.co/models =)




train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▅▄▃▃▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▁▂▁▁▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,638


In [9]:
trainer.model.config.task_specific_params['text-generation'] = {
    'do_sample': True,
    'min_length': 15,
    'max_length': 100,
    'temperature': 100,
    'top_p': 0.95,
    'prefix': '<|endoftext|>'}

In [10]:
#Save Model:
trainer.save_model()

Saving model checkpoint to ./model_files
Configuration saved in ./model_files/config.json
Model weights saved in ./model_files/pytorch_model.bin
tokenizer config file saved in ./model_files/tokenizer_config.json
Special tokens file saved in ./model_files/special_tokens_map.json


In [11]:
a = trainer.state.log_history
print(a[0])

{'loss': 2.9403, 'learning_rate': 0.0001371792092297936, 'epoch': 0.01, 'step': 5}


# Predict:


In [20]:
# prepare input
predictions = []
start = "Mr. Brown,"

"""
Notes:
Temperature: trade-off between variety and politeness clarity
Beam Search vs Top-k/Top-p. Beam search is a lot more coherent with a trade off for variety.

Options:

Naive Beam Search: Num_beams = 10, all else off
Top K with Nucleus Sampling: top_p = 0.95, top_k = 10-20, do_sample=True
Beam-search multinomial sampling : Num_beams = 10 + do_sample = True
Diverse beam-search decoding: Num_beams = 10 + num_beam_groups = 2





"""

start_with_bos = '<|endoftext|>' + start
encoded_prompt = trainer.tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

# prediction
output_sequences = trainer.model.generate(
    input_ids=encoded_prompt,
    max_length=160, #originally 160
    min_length=10, #originally 10
    temperature= 1, #originally 1
    no_repeat_ngram_size=2,    
    
    
    #Edit stuff here down to change decoding 
# BEAM
#     num_beams=10, #on or off

# TOP    
#     do_sample=True, # for multinomial beam search and top sampling
#     top_p = 0.95, #0.95
#     top_k = 50, #10-20

# MULTINOMIAL
#     num_beams=10, #on or off
#     do_sample=True, # for multinomial beam search and top sampling

# DIVERSE 
    num_beams=10, #on or off  
    num_beam_groups = 2, # on or off, must be a multiple of num_beams

    
    
    num_return_sequences= 10, #must = num_beam for diverse beam-search
    
    )
generated_sequences = []

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = trainer.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    if not ALLOW_NEW_LINES:
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

for i, g in enumerate(generated_sequences):
    predictions.append([start, g])

for pair in predictions:
  print(pair[1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/ecbm4040/envTF24/lib/python3.6/site-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I'll get a megaphone like Mr Curry's to remind everyone.
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I suggest preparing some marmalade sandwiches for an extra special elevenses!
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I suggest preparing some marmalade sandwiches for an extra special elevenses!
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I'll get a megaphone like Mr Curry's to remind everyone.
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I'll get a meg megaphone like Mr Curry's to remind everyone.
Mr. Brown, I’m very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I

In [13]:
print("warning")

warning


In [21]:
username = "paddingtonbear" #twitter user
type_tweet = "neutral" #polite, impolite, neutral
decoder = "diverse" #beam, top (top_k + top_p), multinomial (Num_beams + do_sample), diverse (Num_beams + num_beam_groups)

import pandas as pd
df = pd.DataFrame(columns = ["Target","Prompt","Tweets","Type"])
target_col = [username]*len(predictions) # target col
type_col = [type_tweet]*len(predictions) # type col
prompt_col = []
tweets_col = []

for pair in predictions:
    prompt_col.append(pair[0])
    tweets_col.append(pair[1])

df["Target"] = target_col
df["Prompt"] = prompt_col
df["Tweets"] = tweets_col
df["Type"] = type_col
df.reset_index()
# print(df)

df.to_csv('responses/{}_{}_{}.csv'.format(username,type_tweet,decoder), index=False)
#check writing:

df_test = pd.read_csv('responses/{}_{}_{}.csv'.format(username,type_tweet,decoder))
print(df_test)



           Target      Prompt  \
0  paddingtonbear  Mr. Brown,   
1  paddingtonbear  Mr. Brown,   
2  paddingtonbear  Mr. Brown,   
3  paddingtonbear  Mr. Brown,   
4  paddingtonbear  Mr. Brown,   
5  paddingtonbear  Mr. Brown,   
6  paddingtonbear  Mr. Brown,   
7  paddingtonbear  Mr. Brown,   
8  paddingtonbear  Mr. Brown,   
9  paddingtonbear  Mr. Brown,   

                                              Tweets     Type  
0  Mr. Brown, I'm very pleased that you would lik...  neutral  
1  Mr. Brown, I'm very pleased that you would lik...  neutral  
2  Mr. Brown, I'm very pleased that you would lik...  neutral  
3  Mr. Brown, I'm very pleased that you would lik...  neutral  
4  Mr. Brown, I'm very pleased that you would lik...  neutral  
5  Mr. Brown, I’m very pleased that you would lik...  neutral  
6  Mr. Brown, I’m very pleased that you would lik...  neutral  
7  Mr. Brown, I'm very pleased that you would lik...  neutral  
8  Mr. Brown, I'm very pleased that you would lik...  neutra

Empty DataFrame
Columns: [Unnamed: 0, Target, Tweets, Type]
Index: []


# New Section
